<a href="https://colab.research.google.com/github/verma-saloni/Thesis-Work/blob/main/Clean_Fasttext%20With%20SISTER%20workaround.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install unidecode
!pip install word2number
!pip install contractions

In [7]:
import pandas as pd
import numpy as np
import nltk
import gensim
from gensim.models import Word2Vec
import string
string.punctuation
nltk.download('stopwords')
import re
from bs4 import BeautifulSoup
import spacy
import unidecode
from word2number import w2n
import contractions
from nltk.tokenize.treebank import TreebankWordDetokenizer
# load spacy model, can be "en_core_web_sm" as well
from sklearn.model_selection import train_test_split
import xgboost as xgb



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
from google.colab import drive
drive._mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
df_fake = pd.read_csv('/content/drive/MyDrive/Thesis/gossipcop_fake.csv')
df_real = pd.read_csv('/content/drive/MyDrive/Thesis/gossipcop_real.csv')
df_fake['labelML']=0
df_real['labelML']=1

df_fake.count()


id           5323
news_url     5067
title        5323
tweet_ids    5135
labelML      5323
dtype: int64

In [10]:
dforiginal = df_fake.append(df_real)

In [11]:
df= df_fake.append(df_real)
df.reset_index(drop=True, inplace=True)

In [12]:
print(df.columns)
df.count()

Index(['id', 'news_url', 'title', 'tweet_ids', 'labelML'], dtype='object')


id           22140
news_url     21871
title        22140
tweet_ids    20894
labelML      22140
dtype: int64

In [13]:
print(df)

                         id  ... labelML
0      gossipcop-2493749932  ...       0
1      gossipcop-4580247171  ...       0
2       gossipcop-941805037  ...       0
3      gossipcop-2547891536  ...       0
4      gossipcop-5476631226  ...       0
...                     ...  ...     ...
22135      gossipcop-891749  ...       1
22136      gossipcop-941486  ...       1
22137      gossipcop-953143  ...       1
22138      gossipcop-954366  ...       1
22139      gossipcop-877062  ...       1

[22140 rows x 5 columns]


In [14]:
df1= df.drop(['tweet_ids', 'news_url','id'], axis=1)
df1.count()

title      22140
labelML    22140
dtype: int64

In [15]:
# NLP Preprocessing
from gensim.utils import simple_preprocess

# NLP Preprocess - gensim.utils.simple_preprocess(doc, deacc=False, min_len=2, max_len=15)[source]
# Convert a document into a list of tokens.
# This lowercases, tokenizes, de-accents (optional). – the output are final tokens = unicode strings, that won’t be processed any further.

df1.iloc[:, 0] = df1.iloc[:, 0].apply(lambda x: ' '.join(simple_preprocess(x)))

In [16]:
#print(df1)
df1.count()

title      22140
labelML    22140
dtype: int64

In [17]:
# Prefixing each row of the category column with '__label__' 0;10 all rows. same for column. 
df1.iloc[:5322, 0] = df1.iloc[:, 0].apply(lambda x: '__label__0 ' + x)
df1.iloc[5323:, 0] = df1.iloc[1:, 0].apply(lambda x: '__label__1 ' + x)

# NOTE BIG NOTE In general it is important to properly preprocess your data. In particular our example scripts in the root folder do this. i can see why the last cell is wierd, fix in post.


In [18]:
print(df1)

                                                   title  labelML
0      __label__0 did miley cyrus and liam hemsworth ...        0
1      __label__0 paris jackson cara delevingne enjoy...        0
2      __label__0 celebrities join tax march in prote...        0
3      __label__0 cindy crawford daughter kaia gerber...        0
4      __label__0 full list of oscar nominations variety        0
...                                                  ...      ...
22135  __label__1 hollywood film awards the complete ...        1
22136  __label__1 jada pinkett smith explains why son...        1
22137  __label__1 tinsley mortimer reacts to luann de...        1
22138  __label__1 prince harry carries on princess di...        1
22139  __label__1 kylie jenner is actually terrified ...        1

[22140 rows x 2 columns]


In [19]:
df1.count()

title      22140
labelML    22140
dtype: int64

In [20]:
df2=df1
# ------------- df2 = df2.reindex(columns=['labelML', 'title'])
#print(df1)

In [21]:
traindf, testdf = train_test_split(df2, test_size=0.2)
print(traindf)
print(testdf)  

traindf.count()

                                                   title  labelML
8639   __label__1 how jersey shore revival will find ...        1
10475  __label__1 why harry potter fans are freaking ...        1
18942  __label__1 billboard awards jenna dewan domina...        1
1565   __label__0 lady gaga shows off huge engagement...        0
17427  __label__1 bella thorne says scott disick part...        1
...                                                  ...      ...
8059   __label__1 most talked about moments from the ...        1
18665  __label__1 dumbo comes to life in first traile...        1
18486  __label__1 queer eye fans are getting antoni p...        1
14721  __label__1 mystery restaurant accidentally lea...        1
7017   __label__1 tyler perry is donating million to ...        1

[17712 rows x 2 columns]
                                                   title  labelML
12979  __label__1 beyoncé is going vegan again in pre...        1
21061  __label__1 titans trailer robins drops an b

title      17712
labelML    17712
dtype: int64

In [22]:
# Saving the CSV file as a text file to train/test the classifier
import csv #needed
traindf[['labelML', 'title']].to_csv('traindf.txt', 
                                          index = False, 
                                          sep = ' ',
                                          header = None, 
                                          quoting = csv.QUOTE_NONE, 
                                          quotechar = "", 
                                          escapechar = " ")

testdf[['labelML', 'title']].to_csv('testdf.txt', 
                                     index = False, 
                                     sep = ' ',
                                     header = None, 
                                     quoting = csv.QUOTE_NONE, 
                                     quotechar = "", 
                                     escapechar = " ")

In [23]:
!pip install fasttext
import fasttext
# Training the fastText classifier
model = fasttext.train_supervised('traindf.txt', wordNgrams = 2, dim=300)

# Evaluating performance on the entire test file
model.test('testdf.txt')                      

# Predicting on a single input
testdf['labelML'] = testdf['labelML'].astype(str)
model.predict(testdf.iloc[56, 0])
#model.test(testdf.txt)

# Save the trained model
#model.save_model('modelFTme.bin')

(('__label__1',), array([0.67295575]))

In [24]:
testdf.head(10)
#print(testdf.columns)

,title,labelML
12979,__label__1 beyoncé is going vegan again in pre...,1
21061,__label__1 titans trailer robins drops an bomb...,1
2391,__label__0 chloë grace moretz speaks out about...,0
4541,__label__0 brad pitt my twelve years of hell w...,0
9563,__label__1 vanderpump rules finale jax taylor ...,1
16072,__label__1 gigi hadid proves it time to swap y...,1
11272,__label__1 cara delevingne photo gallery high ...,1
6750,__label__1 southern charm shep talks kathryn h...,1
11929,__label__1 chiquis rivera reveals her beauty r...,1
7787,__label__1 latin american music awards ozuna a...,1


This model is running, so trying to change with vectors, or textual input for new vectors. Accuracy is only 50 percent coz I've given it such a balanced dataset.

In [ ]:
# Predicting on a single input
#model.predict(test.iloc['Miley cyrus is new'])
#model.test(testdf.txt)
#use this later when we have the representations I guess. 


In [25]:
# Once the model is trained, we can retrieve the list of words and labels

# model.words 
model.labels
#It returns all words in the vocabulary, sorted by decreasing frequency. 

['__label__1', '__label__0']

In [26]:
# To evaluate our model by computing the precision at 1 (P@1) and the recall on a test set, we use the test function:

def print_results(N, p, r):
    print("N\t" + str(N))
    print("P@{}\t{:.3f}".format(1, p))
    print("R@{}\t{:.3f}".format(1, r))

print_results(*model.test('testdf.txt'))

N	4428
P@1	1.000
R@1	1.000


In [27]:
#We can also predict labels for a specific text. By default, predict returns only one label :
# the one with the highest probability. You can also predict more than one label by specifying the parameter k:
model.predict("katy perry and orlando bloom cozy up", k=2)

(('__label__1', '__label__0'), array([0.67086583, 0.32915416]))

In [28]:
#traindf['title'] = 
model.get_word_vector("the")

array([ 3.63492989e-03,  4.25533652e-02, -3.98730822e-02, -3.82359475e-02,
        2.25766879e-02,  2.71376539e-02, -2.04544291e-02,  3.24283668e-04,
        3.97516824e-02,  5.82379065e-02, -9.29062348e-03,  5.05774096e-02,
       -1.79257076e-02,  4.45502773e-02,  1.74699742e-02,  1.01548294e-02,
       -2.11427622e-02, -9.41488054e-03,  3.43184844e-02, -4.37699072e-03,
        4.07454632e-02,  5.17966561e-02,  1.02073736e-02, -1.41332941e-02,
       -3.34797637e-03,  2.34971335e-03,  2.02158019e-02, -1.79247616e-03,
       -4.54281233e-02,  2.25372594e-02,  7.55718863e-03,  1.57746114e-02,
       -2.39454973e-02,  2.36597285e-02, -2.67151687e-02,  2.91361064e-02,
        1.87356658e-02, -5.00144325e-02, -2.64299251e-02,  5.27520198e-03,
        2.16428880e-02,  1.15506081e-02,  2.97212182e-03, -3.14723216e-02,
       -7.49276194e-04, -2.60525737e-02,  1.49026075e-02, -8.24028812e-03,
        6.34765252e-03,  2.70553157e-02, -4.17404808e-02,  1.19085610e-02,
       -2.58501060e-02, -

In [29]:
model.get_sentence_vector("simon cowell rushed to hospital")

array([-2.0828411e-05,  5.4449625e-03, -5.4275612e-03, -3.9652106e-03,
        3.3909334e-03,  3.5494212e-03, -3.2821349e-03,  4.9510179e-04,
        5.4604923e-03,  7.9394914e-03, -1.1704124e-03,  5.9545189e-03,
       -1.5568185e-03,  6.3504665e-03,  2.4874166e-03,  1.7113602e-03,
       -2.3464574e-03, -3.6827984e-04,  4.4790213e-03, -9.0026343e-04,
        5.0095599e-03,  6.4060730e-03,  6.8410783e-04, -1.2589074e-03,
       -2.2507658e-04, -1.7090159e-04,  1.7470310e-03, -8.0549333e-05,
       -6.0802260e-03,  3.0135456e-03,  1.2908336e-03,  2.3154598e-03,
       -3.2759293e-03,  2.4565023e-03, -3.9448878e-03,  2.7611430e-03,
        1.9411294e-03, -7.0049651e-03, -2.8077886e-03,  7.0554588e-04,
        3.5394451e-03,  1.3703725e-03,  5.4959860e-04, -3.7206437e-03,
       -2.4191073e-04, -3.3507317e-03,  1.9954948e-03,  9.1113172e-05,
        4.2991224e-04,  3.6727984e-03, -5.8367536e-03,  1.5816852e-03,
       -2.9917620e-03, -1.2930275e-03, -8.9254969e-04, -5.0667194e-03,
      

In [30]:
model.get_nearest_neighbors('green') #https://fasttext.cc/docs/en/unsupervised-tutorial.html

[(0.35302436351776123, 'demi'),
 (0.34610652923583984, 'mom'),
 (0.33271485567092896, 'stars'),
 (0.3299618065357208, 'red'),
 (0.3291133642196655, 'netflix'),
 (0.32498109340667725, 'you'),
 (0.31989428400993347, 'mtv'),
 (0.31866654753685, 'bell'),
 (0.3186097741127014, 'brie'),
 (0.3179978132247925, 'see')]

Will be attempting vectors here.

In [31]:
!pip install sister
import sister


In [32]:
embedder = sister.MeanEmbedding(lang="en")

sentence = "I am a dog."
vector = embedder(sentence)  # 300-dim vector


Loading model...


In [33]:
print(vector)

[ 1.45553529e-01  2.93777343e-02  7.25906417e-02 -3.32199186e-01
  1.16119541e-01  3.68818134e-01 -1.01360753e-01  6.13834858e-02
  9.52781364e-02  7.21023753e-02  2.20229030e-02 -1.12615541e-01
  5.77215105e-04  1.48535818e-01 -5.01247048e-02 -2.63762064e-02
 -1.00038767e-01  2.95178946e-02 -4.22474258e-02  6.19036630e-02
  8.73639509e-02  6.23530126e-04 -1.02920629e-01 -5.05673476e-02
 -2.43382186e-01 -5.01833446e-02  3.40687782e-02 -1.88491017e-01
  2.04292275e-02  1.45984618e-02  5.52195907e-02  8.40069652e-02
 -2.68052034e-02  1.21745087e-01 -1.02101944e-01  1.85941588e-02
 -7.98325688e-02 -2.78020445e-02 -2.32372917e-02  8.07202831e-02
  9.49704796e-02 -1.15997776e-01 -5.40836044e-02 -8.32328051e-02
  2.55872514e-02  1.02682985e-01 -1.13008637e-02 -7.92161524e-02
  5.31888008e-03 -5.63134067e-02  1.35553302e-02  2.82942466e-02
 -1.43566862e-01 -5.00163734e-02 -3.94095294e-02  7.77156502e-02
  9.01533887e-02  8.26578513e-02  1.37465566e-01  1.76570296e-01
 -1.79537944e-02  2.28038

In [34]:
dfready = dforiginal.drop(['tweet_ids', 'news_url','id'], axis=1)

from string import punctuation
from nltk.corpus import stopwords




In [35]:
def preprocess(data):
    """
    Preprocess data, filtering out stopwords, punctuation and lowering 
    all splitted tokens.
    
    :param data: the string data to be processed
    """    
    # Pad punctuation with spaces on both sides
    for char in ['.', '"', ',', '(', ')', '!', '?', ';', ':']:
        data = data.replace(char, ' ' + char + ' ')
    sw = stopwords.words('english')
    splitted_chunks = data.split()
    lowered_chunks = (item.lower() for item in splitted_chunks)
    chunks_without_punctuation = (chunk for chunk in lowered_chunks if chunk not in punctuation)
    chunks_without_stopwords = (chunk for chunk in chunks_without_punctuation if chunk not in sw)
    return " ".join(list(chunks_without_stopwords))


In [ ]:
import pandas as pd
def write_data(filename, data):
    """
    Write result to a file.
    
    :param result: the list to be written to the file
    """
    with open(filename, "a") as output:
        output.write('{}\n'.format(data))

def create_vectors(title):
  embedder = sister.MeanEmbedding(lang="en")
  sentence = title
  vector = embedder(sentence)  # 300-dim vector
  return vector

# executing the function
dfready['title'] = dfready['title'].apply(preprocess)

# executing the function
dfready['title'] = dfready['title'].apply(create_vectors)

dfready.preprocess()
dfready.create_vectors()

Loading model...
Loading model...


Loading model...


Loading model...


Loading model...


Loading model...


...

Loading model...
Loading model...


Loading model...
Loading model...


Loading model...
Loading model...


Loading model...


Loading model...


In [3]:
dfready.head()

NameError: ignored

In [ ]:
import xgboost as xgb
#TASK #2: Create an XGBoost classifier that will work for the vectors that you have (size 300), and train it....

# Instantiate the XGBClassifier: xg_cl
xg_cl = xgb.XGBClassifier(objective='binary:logistic', n_estimators=10, seed=123)

# Fit the classifier to the training set
xg_cl.fit(X_train, y_train)

# Predict the labels of the test set: preds
preds = xg_cl.predict(X_test)

# Compute the accuracy: accuracy
accuracy = float(np.sum(preds == y_test)) / y_test.shape[0]
print("accuracy: %f" % (accuracy))

#TASK 3: Analyze the results of one training example with the confusion matrix

#If you can also get the confusion matrix

accuracy: 1.000000
